### Find Ngram Youtube Link

In [51]:
import os
import pickle
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index

# youtube
sample_num = 10  # 7
time_shift = 0.6

# main or additional result
main_search = True  # False for additional result

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].sample(sample_num)
            #df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].tail(sample_num)  # will test
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [7]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [8]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/3-Find Ngram Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

In [9]:
## Master File
#path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 5/Data/Deployment/Talk Time 5 Master File.xlsx"
#sheet = "Sheet6"  # Sheet1
#file_ext = "Sixgram"

In [61]:
# search file path folder
ngram = "Fourgram"
path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Data/\
Manuel Select/Revised Ngrams All in One.xlsx"
sheet = "Sheet2"
file_ext = ngram
# youtube tool output path
out_path = "/home/kurubal/Downloads"

#### Read Data File

In [62]:
if main_search:
    df_file = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
else:
    df_file = pd.read_excel(f"{out_path}/{ngram}_Re-Search_Result.xlsx")
    
df_file

,fourgram,threegram,twogram
0,bana yardım eder misin,NaN,NaN
1,bana yardım eder misin,NaN,NaN
2,bana yardım eder misin,yardım eder misin,eder misin
3,bana yardım eder misin,yardım eder misin,yardım eder
4,başka bir şey daha,başka bir şey,bir şey
...,...,...,...
495,NaN,çok kötü bir,çok kötü
496,NaN,bir şey olmaz,NaN
497,NaN,şuna bir bak,bir bak
498,NaN,şuna bir bak,NaN


In [63]:
if main_search:
    search_list = list(set(df_file.loc[:,f"{ngram.lower()}"].dropna()))
else:
    search_list = list(set(df_file["not_result"]))
    
len(search_list)

100

In [55]:
if main_search:
    disable_video_id_list = []
else:
    with open("disable_videoid.pkl", "rb") as file:
        disable_video_id_list = pickle.load(file)

In [65]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [66]:
#sent_count = len(df_youtube_sentence)
#sent_count

3934207

In [67]:
#df_youtube_sentence = df_youtube_sentence.sample(sent_count, random_state=52)
#df_youtube_sentence.reset_index(drop=True, inplace=True)
#df_youtube_sentence

,start_time,end_time,sentence,video_id
0,05:16:31.048,05:16:33.248,i drak edecek nice sırları var,kKPqeq4IrB4
1,00:21:15.700,00:21:17.500,direkt uçağa biniyoruz,akzpB69pSwk
2,00:00:02.242,00:00:03.397,kanalıma hoş geldiniz,rKf_XVvbuDU
3,00:04:42.820,00:04:47.980,merhaba nasılsın i yiyim teşekkürler siz i yiy...,j0JPiHrd55M
4,00:29:21.840,00:29:22.720,ben çağırıyorum,pvsImT6E4yM
...,...,...,...,...
3934202,00:34:48.020,00:34:50.562,smoke was coming out of the wounds,z7SwoO1gIs4
3934203,00:02:55.450,00:02:55.480,bu,5Z8oBJZSBMY
3934204,00:01:17.056,00:01:19.860,bu da testi çok daha zorlayıcı hale getirip,t0rtkI8P8kY
3934205,02:00:25.698,02:00:27.767,var nedir,XNBOk8QO8I0


In [68]:
df_youtube_sentence = df_youtube_sentence[~df_youtube_sentence["video_id"].isin(disable_video_id_list)]
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,05:16:31.048,05:16:33.248,i drak edecek nice sırları var,kKPqeq4IrB4
1,00:21:15.700,00:21:17.500,direkt uçağa biniyoruz,akzpB69pSwk
2,00:00:02.242,00:00:03.397,kanalıma hoş geldiniz,rKf_XVvbuDU
3,00:04:42.820,00:04:47.980,merhaba nasılsın i yiyim teşekkürler siz i yiy...,j0JPiHrd55M
4,00:29:21.840,00:29:22.720,ben çağırıyorum,pvsImT6E4yM
...,...,...,...,...
3934202,00:34:48.020,00:34:50.562,smoke was coming out of the wounds,z7SwoO1gIs4
3934203,00:02:55.450,00:02:55.480,bu,5Z8oBJZSBMY
3934204,00:01:17.056,00:01:19.860,bu da testi çok daha zorlayıcı hale getirip,t0rtkI8P8kY
3934205,02:00:25.698,02:00:27.767,var nedir,XNBOk8QO8I0


In [69]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [70]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,18991.048,18993.248,i drak edecek nice sırları var,kKPqeq4IrB4
1,1275.700,1277.500,direkt uçağa biniyoruz,akzpB69pSwk
2,2.242,3.397,kanalıma hoş geldiniz,rKf_XVvbuDU
3,282.820,287.980,merhaba nasılsın i yiyim teşekkürler siz i yiy...,j0JPiHrd55M
4,1761.840,1762.720,ben çağırıyorum,pvsImT6E4yM
...,...,...,...,...
3934202,2088.020,2090.562,smoke was coming out of the wounds,z7SwoO1gIs4
3934203,175.450,175.480,bu,5Z8oBJZSBMY
3934204,77.056,79.860,bu da testi çok daha zorlayıcı hale getirip,t0rtkI8P8kY
3934205,7225.698,7227.767,var nedir,XNBOk8QO8I0


#### Find Ngram In Youtube

In [71]:
if main_search:
    df_word_group = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
else:
    df_word_group = word_group_youtube(df_youtube_sentence, search_list, "sentence", 30)
    
df_word_group

,search_string,start_time,end_time,sentence,video_id
0,bu da ne böyle,2380.164,2381.521,bu da ne böyle,_srfsUOpKig
1,bu da ne böyle,1495.600,1496.720,bu da ne böyle,oD7GlhIvAlM
2,bu da ne böyle,2769.987,2771.916,bu da ne böyle,3sqBlmCqhpY
3,bu da ne böyle,962.700,964.680,aaa a bu da ne böyle,82S9KcNuZes
4,bu da ne böyle,3364.584,3365.893,bu da ne böyle,bsQD5xMYc7o
...,...,...,...,...,...
934,ne fark eder ki,1696.360,1701.020,ya ha kırk ha seksen yani ne fark eder ki haya...,Z4l2qH8sRIc
935,ne fark eder ki,9055.291,9060.133,güzelmiş çirkinmiş ne fark eder ki,mpg2dM5DrGg
936,ne fark eder ki,3942.797,3944.507,öyle de ne fark eder ki,0Rtmbtf7piU
937,ne fark eder ki,8347.303,8352.336,güzelmiş çirkinmiş ne fark eder ki,x6aEnqpTUBo


In [72]:
df_word_group_time_loc = word_group_time_loc(df_word_group, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,bu da ne böyle,2380.164000,2381.521000,bu da ne böyle,_srfsUOpKig
1,bu da ne böyle,1495.600000,1496.720000,bu da ne böyle,oD7GlhIvAlM
2,bu da ne böyle,2769.987000,2771.916000,bu da ne böyle,3sqBlmCqhpY
3,bu da ne böyle,963.195000,964.680000,aaa a bu da ne böyle,82S9KcNuZes
4,bu da ne böyle,3364.584000,3365.893000,bu da ne böyle,bsQD5xMYc7o
...,...,...,...,...,...
934,ne fark eder ki,1698.239032,1699.516774,ya ha kırk ha seksen yani ne fark eder ki haya...,Z4l2qH8sRIc
935,ne fark eder ki,9057.854412,9060.133000,güzelmiş çirkinmiş ne fark eder ki,mpg2dM5DrGg
936,ne fark eder ki,3943.317435,3944.507000,öyle de ne fark eder ki,0Rtmbtf7piU
937,ne fark eder ki,8349.967529,8352.336000,güzelmiş çirkinmiş ne fark eder ki,x6aEnqpTUBo


In [73]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,bu da ne böyle,2379.564000,2382.121000,bu da ne böyle,_srfsUOpKig
1,bu da ne böyle,1495.000000,1497.320000,bu da ne böyle,oD7GlhIvAlM
2,bu da ne böyle,2769.387000,2772.516000,bu da ne böyle,3sqBlmCqhpY
3,bu da ne böyle,962.595000,965.280000,aaa a bu da ne böyle,82S9KcNuZes
4,bu da ne böyle,3363.984000,3366.493000,bu da ne böyle,bsQD5xMYc7o
...,...,...,...,...,...
934,ne fark eder ki,1697.639032,1700.116774,ya ha kırk ha seksen yani ne fark eder ki haya...,Z4l2qH8sRIc
935,ne fark eder ki,9057.254412,9060.733000,güzelmiş çirkinmiş ne fark eder ki,mpg2dM5DrGg
936,ne fark eder ki,3942.717435,3945.107000,öyle de ne fark eder ki,0Rtmbtf7piU
937,ne fark eder ki,8349.367529,8352.936000,güzelmiş çirkinmiş ne fark eder ki,x6aEnqpTUBo


In [74]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: round(x))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: round(x))
df_word_group_time_loc 

,search_string,start_time,end_time,sentence,video_id
0,bu da ne böyle,2380,2382,bu da ne böyle,_srfsUOpKig
1,bu da ne böyle,1495,1497,bu da ne böyle,oD7GlhIvAlM
2,bu da ne böyle,2769,2773,bu da ne böyle,3sqBlmCqhpY
3,bu da ne böyle,963,965,aaa a bu da ne böyle,82S9KcNuZes
4,bu da ne böyle,3364,3366,bu da ne böyle,bsQD5xMYc7o
...,...,...,...,...,...
934,ne fark eder ki,1698,1700,ya ha kırk ha seksen yani ne fark eder ki haya...,Z4l2qH8sRIc
935,ne fark eder ki,9057,9061,güzelmiş çirkinmiş ne fark eder ki,mpg2dM5DrGg
936,ne fark eder ki,3943,3945,öyle de ne fark eder ki,0Rtmbtf7piU
937,ne fark eder ki,8349,8353,güzelmiş çirkinmiş ne fark eder ki,x6aEnqpTUBo


In [75]:
#df_word_group_time_loc["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc['video_id'].map(str)+"&t="+df_word_group_time_loc['start_time'].map(str)+"s"
#df_word_group_time_loc

In [76]:
word_count_result(df_word_group_time_loc, ["search_string"], set_condition=True)

,word,word_count
0,bir,46
1,şey,39
2,ne,23
3,var,18
4,mi,12
...,...,...
115,altında,1
116,seviyorum,1
117,fark,1
118,istediğim,1


In [77]:
if main_search:
    df_word_group_time_loc.to_excel(f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link.xlsx", index=False)  ########*****
else:
    df_word_group_time_loc.to_excel(f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link2.xlsx", index=False)

#### Copy Move And Delete

In [78]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_With_{word_end}_Word_Youtube_Link*.xlsx")   ########*****
output_file

['Turkish_Twogram_With_1000_Word_Youtube_Link.xlsx',
 'Turkish_Threegram_With_1000_Word_Youtube_Link.xlsx',
 'Turkish_Fourgram_With_1000_Word_Youtube_Link.xlsx']

In [79]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [80]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Tool

In [56]:
path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Data/\
Manuel Select/Revised Ngrams All in One.xlsx"
sheet = "Sheet2"

In [57]:
df_file = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_file = pd.read_excel(f"twogram.xlsx")
df_file

,fourgram,threegram,twogram
0,bana yardım eder misin,NaN,NaN
1,bana yardım eder misin,NaN,NaN
2,bana yardım eder misin,yardım eder misin,eder misin
3,bana yardım eder misin,yardım eder misin,yardım eder
4,başka bir şey daha,başka bir şey,bir şey
...,...,...,...
495,NaN,çok kötü bir,çok kötü
496,NaN,bir şey olmaz,NaN
497,NaN,şuna bir bak,bir bak
498,NaN,şuna bir bak,NaN


In [58]:
if main_search:
    df_twogram_link = pd.read_excel(f"{path}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_Youtube_Link.xlsx")
else:
    df_twogram_link = pd.read_excel(f"{path}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_Youtube_Link2.xlsx")
    
df_twogram_link

,search_string,start_time,end_time,sentence,video_id
0,demek istiyorsun,5093,5095,ne demek istiyorsun kumandan,lFWYh_HTYxE
1,demek istiyorsun,2440,2442,ne demek istiyorsun,ne_t7rBCcDw
2,demek istiyorsun,2621,2624,sen şimdi bana küçük meselelerin adamısın mı d...,oPIxVs5VmUg
3,demek istiyorsun,1049,1051,dayak yedim mi demek istiyorsun,j5XOzTRnvjg
4,demek istiyorsun,4772,4775,i çime sığmayan derken neler demek istiyorsun ...,Iw1u6ZYA4t8
...,...,...,...,...,...
1772,daha önce,4923,4925,ben hayatımda daha önce hiç mum üflemedim ya,Yr2ZLfWT0i8
1773,daha önce,13,15,doğal düşün tvye hoşgeldiniz bu programda daha...,w312GmKmLFk
1774,daha önce,399,401,olduğunu düşündüğünüz şey bile daha önce çok k...,7HrxKm4FDw8
1775,daha önce,2307,2308,daha önce ne yapıyordu ne ediyordu hiç bilmiyorum,EyxjFfK2GWA


In [59]:
set(df_file["twogram"].dropna()).difference(set(df_twogram_link["search_string"]))

set()

In [60]:
if main_search:
    df_threegram_link = pd.read_excel(f"{path}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_Youtube_Link.xlsx")
else:
    df_threegram_link = pd.read_excel(f"{path}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_Youtube_Link2.xlsx")
    
df_threegram_link

,search_string,start_time,end_time,sentence,video_id
0,hiçbir fikrim yok,30,32,genelde öncesinde deneyimliyorum neden böyle o...,7L64XN3or3I
1,hiçbir fikrim yok,231,233,merdiveni galiba biz bunları nasıl birleştirec...,yfbaI_fKgQc
2,hiçbir fikrim yok,2760,2762,nedense oğlum hakkında neler olduğuna dair hiç...,6GC8ekmdngk
3,hiçbir fikrim yok,129,131,hiçbir fikrim yok,iPtFF0mw5_Y
4,hiçbir fikrim yok,3957,3960,hiçbir fikrim yok hocam,a5oQSIf9isU
...,...,...,...,...,...
1732,iyi bir fikir,7343,7345,bak bunun pek iyi bir fikir olduğunu düşünmüyorum,SjVWia8J9JI
1733,iyi bir fikir,965,967,ne diyorsun meryem çok iyi bir fikir,HbdYlwdvXmU
1734,iyi bir fikir,98,100,herkesin hokeye katılması gibi iyi bir fikir b...,oAbzIuXYrgM
1735,iyi bir fikir,837,840,mesela prens harrynin bir kostümlü partiye naz...,C89r72cHxm8


In [61]:
set(df_file["threegram"].dropna()).difference(set(df_threegram_link["search_string"]))

{'da neydi böyle'}

In [62]:
if main_search:
    df_fourgram_link = pd.read_excel(f"{path}/{lang_folder.capitalize()}_Fourgram_With_{word_end}_Word_Youtube_Link.xlsx")
else:
    df_fourgram_link = pd.read_excel(f"{path}/{lang_folder.capitalize()}_Fourgram_With_{word_end}_Word_Youtube_Link2.xlsx")
    
df_fourgram_link

,search_string,start_time,end_time,sentence,video_id
0,bu da ne böyle,2380,2382,bu da ne böyle,_srfsUOpKig
1,bu da ne böyle,1495,1497,bu da ne böyle,oD7GlhIvAlM
2,bu da ne böyle,2769,2773,bu da ne böyle,3sqBlmCqhpY
3,bu da ne böyle,963,965,aaa a bu da ne böyle,82S9KcNuZes
4,bu da ne böyle,3364,3366,bu da ne böyle,bsQD5xMYc7o
...,...,...,...,...,...
934,ne fark eder ki,1698,1700,ya ha kırk ha seksen yani ne fark eder ki haya...,Z4l2qH8sRIc
935,ne fark eder ki,9057,9061,güzelmiş çirkinmiş ne fark eder ki,mpg2dM5DrGg
936,ne fark eder ki,3943,3945,öyle de ne fark eder ki,0Rtmbtf7piU
937,ne fark eder ki,8349,8353,güzelmiş çirkinmiş ne fark eder ki,x6aEnqpTUBo


In [63]:
set(df_file["fourgram"].dropna()).difference(set(df_fourgram_link["search_string"]))

{'bu da neydi böyle'}

In [52]:
disable_videoid = list(set(df_twogram_link["video_id"])) + list(set(df_threegram_link["video_id"])) + list(set(df_fourgram_link["video_id"]))
#disable_videoid

In [53]:
if main_search:
    with open("disable_videoid.pkl", "wb") as file:
        pickle.dump(disable_videoid, file)
else:
    pass 

In [19]:
df_twogram_link_reassign = df_twogram_link.copy()
df_threegram_link_reassign = df_threegram_link.copy()
df_fourgram_link_reassign = df_fourgram_link.copy()

In [20]:
df_twogram_link_reassign["start_time"] = 1
df_twogram_link_reassign["end_time"] = 2
df_threegram_link_reassign["start_time"] = 1
df_threegram_link_reassign["end_time"] = 2
df_fourgram_link_reassign["start_time"] = 1
df_fourgram_link_reassign["end_time"] = 2

In [22]:
df_twogram_link_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_Youtube_Link_Temp.xlsx", index=False)
df_threegram_link_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_Youtube_Link_Temp.xlsx", index=False)
df_fourgram_link_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Fourgram_With_{word_end}_Word_Youtube_Link_Temp.xlsx", index=False)

In [64]:
# read file after youtube tool result
df_twogram_link_reassign_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_Youtube_Link_Temp.xlsx")  # from youtube tool
df_twogram_link_reassign_video_list = list(set(df_twogram_link_reassign_youtube_tool["video_id"]))

In [ ]:
df_threegram_link_reassign_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_Youtube_Link_Temp.xlsx")  # from youtube tool
df_threegram_link_reassign_video_list = list(set(df_threegram_link_reassign_youtube_tool["video_id"]))

In [24]:
df_fourgram_link_reassign_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Fourgram_With_{word_end}_Word_Youtube_Link_Temp.xlsx")  # from youtube tool
df_fourgram_link_reassign_video_list = list(set(df_fourgram_link_reassign_youtube_tool["video_id"]))

In [66]:
df_twogram_link_new = df_twogram_link[df_twogram_link["video_id"].isin(df_twogram_link_reassign_video_list)]
df_twogram_link_new

,search_string,start_time,end_time,sentence,video_id
16,duyuyor musun,1073,1075,abone ol var ya yine gitti oraya taktı ya mehm...,NtoRI5nlwKw
30,güzel bir,153,155,güzel bir kahvaltı yaptım bu aldığım kaloriler...,CwuGepQDBrM
31,güzel bir,941,944,demek ki ne yapıyorsunuz buraya piknik sepetin...,UQDUQnVUtq4
33,güzel bir,98,100,hazırlayabileceği çok güzel bir ekmek,dRCz3FhBk08
34,güzel bir,13572,13574,s s n yan sülalesi huzurlarında güzel bir söz ...,7XVwNBlTIWs
...,...,...,...,...,...
1770,daha önce,157,159,çünkü kardeşim daha önce kantincilik yaptığı i...,XjIzyvw_UMA
1771,daha önce,178,179,ilk kez böyle bir şey başıma geldi daha önce ş...,OAHJQ3_lKds
1772,daha önce,4923,4925,ben hayatımda daha önce hiç mum üflemedim ya,Yr2ZLfWT0i8
1773,daha önce,13,15,doğal düşün tvye hoşgeldiniz bu programda daha...,w312GmKmLFk


In [67]:
df_twogram_not_result = pd.DataFrame(set(df_file["twogram"].dropna()).difference(set(df_twogram_link_new["search_string"])), columns=["not_result"])
df_twogram_not_result

,not_result
0,yaptığını sanıyorsun
1,kadar yeter
2,biliyor musun
3,nerede olduğunu
4,her şeyi
5,ne dedin
6,söyledi mi
7,ne düşünüyorum
8,fikir değil
9,iyi misin


In [25]:
df_threegram_link_new = df_threegram_link[df_threegram_link["video_id"].isin(df_threegram_link_reassign_video_list)]
df_threegram_link_new

,search_string,start_time,end_time,sentence,video_id
0,hiçbir fikrim yok,30,32,genelde öncesinde deneyimliyorum neden böyle o...,7L64XN3or3I
1,hiçbir fikrim yok,231,233,merdiveni galiba biz bunları nasıl birleştirec...,yfbaI_fKgQc
5,hiçbir fikrim yok,461,464,ama şu anda hiçbir fikrim yok,OtTPQ4877Hk
7,hiçbir fikrim yok,371,374,hiçbir fikrim yok,xagFSGk8vT4
8,hiçbir fikrim yok,384,386,hiçbir fikrim yok,L3bf7dlR5hk
...,...,...,...,...,...
1722,için çok teşekkür,569,571,hepinize izlediğiniz için çok teşekkür,8iUkBU1vsDw
1723,için çok teşekkür,435,438,hik yemi dinlediğiniz için çok teşekkür ederim,fvuHfBDV7DU
1731,iyi bir fikir,340,342,yine çok iyi bir fikir mi şu an emin değilim,KSs8rK9tO04
1734,iyi bir fikir,98,100,herkesin hokeye katılması gibi iyi bir fikir b...,oAbzIuXYrgM


In [48]:
df_threegram_not_result = pd.DataFrame(set(df_file["threegram"].dropna()).difference(set(df_threegram_link_new["search_string"])), columns=["not_result"])
df_threegram_not_result

,not_result
0,sorun mu var
1,önce hiç görmedim
2,beni duyuyor musun
3,ne yaptığını sanıyorsun
4,bir sorun mu
5,şey iyi olacak
6,bir şey ister
7,daha ne kadar
8,da ne böyle
9,bir şey olmayacak


In [26]:
df_fourgram_link_new = df_fourgram_link[df_fourgram_link["video_id"].isin(df_fourgram_link_reassign_video_list)]
df_fourgram_link_new

,search_string,start_time,end_time,sentence,video_id
8,neden bu kadar çok,74,76,underrated bir dizi ve ben bu diziyi neden bu ...,3cAaBZ_Aml8
9,neden bu kadar çok,26,28,sizce bunun asıl nedenleri nedir neden bu kada...,yOjfgHwn9y8
11,neden bu kadar çok,246,248,neden bu kadar çok okunmuştu,yK7PsjTRfwY
12,neden bu kadar çok,514,517,neden bu kadar çok bu işi şey yapıyorsun önems...,qaZnDHacYSk
16,neden bu kadar çok,981,983,neden bu kadar çok ilgi görmüyor veya,WM2W7dCq3tA
...,...,...,...,...,...
892,bunu daha önce de,17,19,bunu daha önce de paylaştım,VmFZ5g6Zeyw
895,başka bir şey yok,458,461,söylüyoruz evet değiştiriyoruz başka bir şey y...,C6NzdthV39E
900,başka bir şey yok,1774,1775,tamam hadi başka bir şey yok bunu söylemek içi...,66Oih2dmURc
920,böyle bir şey olmayacak,4105,4108,herkes onun dediğini dinlesin isterler ama böy...,BfX0BJRrRhI


In [41]:
df_fourgram_not_result = pd.DataFrame(set(df_file["fourgram"].dropna()).difference(set(df_fourgram_link_new["search_string"])), columns=["not_result"])
df_fourgram_not_result

,not_result
0,bir şey mi var
1,bu da neydi böyle
2,önemli bir şey değil
3,şaka yapıyorsun değil mi
4,burada ne işi var
5,sakin ol tamam mı
6,sizin için ne yapabilirim
7,için her şeyi yaparım
8,nerede olduğunu biliyor musun
9,sana yardım etmek istiyorum


In [68]:
df_twogram_link_new.to_excel(f"{out_path}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_Youtube_Link.xlsx", index=False)
df_threegram_link_new.to_excel(f"{out_path}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_Youtube_Link.xlsx", index=False)
df_fourgram_link_new.to_excel(f"{out_path}/{lang_folder.capitalize()}_Fourgram_With_{word_end}_Word_Youtube_Link.xlsx", index=False)

#### Research New

In [ ]:
df_twogram_link_new = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_Youtube_Link.xlsx")
df_twogram_link_new

In [73]:
df_twogram_count = df_twogram_link_new.groupby(["search_string"])[["search_string"]].count()
df_twogram_count.rename(columns={"search_string":"count"}, inplace=True)
df_twogram_count.sort_values(by="count", ascending=False, inplace=True)
df_twogram_count.reset_index(inplace=True)
df_twogram_count

,search_string,count
0,çok büyük,9
1,için teşekkürler,8
2,herhangi bir,8
3,söylemek istiyorum,8
4,bu çok,8
...,...,...
157,yok değil,1
158,ne demek,1
159,zorunda değilsin,1
160,işin var,1


In [74]:
df_twogram_additional = df_twogram_count[df_twogram_count["count"] < 3]
df_twogram_additional = df_twogram_additional.rename(columns={"search_string":"not_result"})
df_twogram_additional.drop("count", axis=1, inplace=True)
df_twogram_additional

,not_result
112,sağ ol
113,ben de
114,bir fikrim
115,bir şey
116,bu doğru
117,bunu biliyorsun
118,o kadar
119,fikrim yok
120,ne olduğunu
121,iyi bak


In [75]:
df_twogram_new_search = pd.concat([df_twogram_not_result, df_twogram_additional], axis=0)
df_twogram_new_search

,not_result
0,yaptığını sanıyorsun
1,kadar yeter
2,biliyor musun
3,nerede olduğunu
4,her şeyi
...,...
157,yok değil
158,ne demek
159,zorunda değilsin
160,işin var


In [76]:
df_twogram_new_search.to_excel(f"{out_path}/Twogram_Re-Search_Result.xlsx", index=False)

In [ ]:
df_threegram_link_new = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_Youtube_Link.xlsx")
df_threegram_link_new

In [69]:
df_threegram_count = df_threegram_link_new.groupby(["search_string"])[["search_string"]].count()
df_threegram_count.rename(columns={"search_string":"count"}, inplace=True)
df_threegram_count.sort_values(by="count", ascending=False, inplace=True)
df_threegram_count.reset_index(inplace=True)
df_threegram_count

,search_string,count
0,ve bu da,9
1,görmek çok güzel,8
2,çok özür dilerim,8
3,bir tane daha,8
4,bu doğru mu,7
...,...,...
143,her şey için,1
144,kimse var mı,1
145,yardım etmek istiyorum,1
146,kimse yok mu,1


In [70]:
df_threegram_additional = df_threegram_count[df_threegram_count["count"] < 3]
df_threegram_additional = df_threegram_additional.rename(columns={"search_string":"not_result"})
df_threegram_additional.drop("count", axis=1, inplace=True)
df_threegram_additional

,not_result
84,uzun zaman oldu
85,bir daha asla
86,sen ve ben
87,yardımcı olabilir miyim
88,peki ya sen
...,...
143,her şey için
144,kimse var mı
145,yardım etmek istiyorum
146,kimse yok mu


In [71]:
df_threegram_new_search = pd.concat([df_threegram_not_result, df_threegram_additional], axis=0)
df_threegram_new_search

,not_result
0,sorun mu var
1,önce hiç görmedim
2,beni duyuyor musun
3,ne yaptığını sanıyorsun
4,bir sorun mu
...,...
143,her şey için
144,kimse var mı
145,yardım etmek istiyorum
146,kimse yok mu


In [72]:
df_threegram_new_search.to_excel(f"{out_path}/Threegram_Re-Search_Result.xlsx", index=False)

In [ ]:
df_fourgram_link_new = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Fourgram_With_{word_end}_Word_Youtube_Link.xlsx")
df_fourgram_link_new

In [77]:
df_fourgram_count = df_fourgram_link_new.groupby(["search_string"])[["search_string"]].count()
df_fourgram_count.rename(columns={"search_string":"count"}, inplace=True)
df_fourgram_count.sort_values(by="count", ascending=False, inplace=True)
df_fourgram_count.reset_index(inplace=True)
df_fourgram_count

,search_string,count
0,seni görmek çok güzel,10
1,onun gibi bir şey,7
2,bir şey söylemek istiyorum,7
3,bu da ne demek,6
4,ne olduğunu biliyor musun,6
...,...,...
66,bir şey söyledi mi,1
67,bir şey duydun mu,1
68,ben de seni seviyorum,1
69,başka bir şey var,1


In [78]:
df_fourgram_additional = df_fourgram_count[df_fourgram_count["count"] < 3]
df_fourgram_additional = df_fourgram_additional.rename(columns={"search_string":"not_result"})
df_fourgram_additional.drop("count", axis=1, inplace=True)
df_fourgram_additional

,not_result
38,evet mi hayır mı
39,bunu sen mi yaptın
40,ne var biliyor musun
41,bu ne demek oluyor
42,kötü bir şey mi
43,başka bir şey yok
44,bunu daha önce de
45,sadece sen ve ben
46,böyle bir şey olmayacak
47,hiç böyle bir şey


In [79]:
df_fourgram_new_search = pd.concat([df_fourgram_not_result, df_fourgram_additional], axis=0)
df_fourgram_new_search

,not_result
0,bir şey mi var
1,bu da neydi böyle
2,önemli bir şey değil
3,şaka yapıyorsun değil mi
4,burada ne işi var
...,...
66,bir şey söyledi mi
67,bir şey duydun mu
68,ben de seni seviyorum
69,başka bir şey var


In [80]:
df_fourgram_new_search.to_excel(f"{out_path}/Fourgram_Re-Search_Result.xlsx", index=False)